# Devlog 2023-07-13

Now that we have done a compatibility pass with all of the system modules, let's do a compatibility test. The results of this test should comport with [the analysis in this spreadsheet](https://docs.google.com/spreadsheets/d/1FF_ovXYQUb65JagcLj8QCmCzs2_Pzd9qnZ5Cr28v4jw/edit?usp=sharing).

Generate the cross product of all available modules and test them one-by-one to see if they run successfully. We'll just run each simulation for a few days and verify that this doesn't throw any exceptions. Ideally, these results will line up with our expected compatibility matrix.

_Note: also requires the `seaborn` library._

In [2]:
from datetime import date
import time
from typing import Callable
from epymorph.data import mm_library, geo_library, ipm_library
from itertools import product
import pandas as pd
from epymorph.geo.geo import Geo
from epymorph.movement.basic import BasicEngine

from epymorph.simulation import Simulation

geo_order = ['pei', 'single_pop', 'us_counties_2015',
             'us_states_2015', 'us_sw_counties_2015', 'maricopa_cbg_2019']
mm_order = ['pei', 'sparsemod', 'centroids', 'icecube', 'no']
ipm_order = ['pei', 'sirs', 'sirh', 'sparsemod', 'no']

# pre-load the geos, otherwise we'll waste time reloading these for every simulation
cached_geos = {
    name: geo_library[name]()
    for name in geo_order
}

combos = pd.DataFrame(
    product(geo_order, mm_order, ipm_order),
    columns=['geo', 'mm', 'ipm']
)

# Params to satisfy (almost) every combo!
params = {
    'alpha': [0.05, 0.4125, 0.5875],
    'beta': 0.4,
    'gamma': 0.1000022,
    'xi': 1 / 90.0,
    'phi': 40,
    'theta': 0.1,
    'move_control': 0.9,
    'infection_duration': 4.0,
    'immunity_duration': 90.0,
    'hospitalization_duration': 14.0,
    'hospitalization_rate': 0.1,
    # initializer: single_location
    'location': 0,
    'seed_size': 100
}

sparsemod_ipm_params = {
    'omega': [0.55, 0.05],
    'delta': [0.333, 0.5, 0.166, 0.142, 0.125],
    'gamma': [0.166, 0.333, 0.25],
    'rho': [0.40, 0.175, 0.015, 0.20, 0.60],
}


def test(geo, mm, ipm):
    success = True
    t0 = time.perf_counter()
    try:
        if ipm == 'sparsemod':
            ps = params | sparsemod_ipm_params
        else:
            ps = params

        Simulation(
            geo=cached_geos[geo],
            ipm_builder=ipm_library[ipm](),
            mvm_builder=mm_library[mm](),
            mvm_engine=BasicEngine
        ).run(ps, date(2010, 1, 1), 3)
    except:
        success = False
    t1 = time.perf_counter()
    print(f"{'succeeded' if success else 'failed'}: {geo, mm, ipm} in {((t1 - t0) * 1000.0):.3f} ms")
    return pd.Series([success, t1 - t0])


compat = combos.copy()
compat[['runs', 'runtime']] = combos.apply(
    lambda x: test(x['geo'], x['mm'], x['ipm']),
    axis=1)

succeeded: ('pei', 'pei', 'pei') in 68.934 ms
succeeded: ('pei', 'pei', 'sirs') in 22.465 ms
succeeded: ('pei', 'pei', 'sirh') in 128.177 ms
succeeded: ('pei', 'pei', 'sparsemod') in 112.438 ms
succeeded: ('pei', 'pei', 'no') in 14.759 ms
succeeded: ('pei', 'sparsemod', 'pei') in 129.578 ms
succeeded: ('pei', 'sparsemod', 'sirs') in 19.034 ms
succeeded: ('pei', 'sparsemod', 'sirh') in 25.095 ms
succeeded: ('pei', 'sparsemod', 'sparsemod') in 68.769 ms
succeeded: ('pei', 'sparsemod', 'no') in 7.994 ms
succeeded: ('pei', 'centroids', 'pei') in 116.960 ms
succeeded: ('pei', 'centroids', 'sirs') in 20.914 ms
succeeded: ('pei', 'centroids', 'sirh') in 24.693 ms
succeeded: ('pei', 'centroids', 'sparsemod') in 69.013 ms
succeeded: ('pei', 'centroids', 'no') in 7.933 ms
succeeded: ('pei', 'icecube', 'pei') in 123.715 ms
succeeded: ('pei', 'icecube', 'sirs') in 17.383 ms
succeeded: ('pei', 'icecube', 'sirh') in 22.840 ms
succeeded: ('pei', 'icecube', 'sparsemod') in 66.842 ms
succeeded: ('pei',

In [ ]:
from functools import partial, reduce
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

boolean_or = partial(reduce, bool.__or__)


mm_compat = compat.groupby(by=['geo', 'mm'])\
    .agg({'runs': boolean_or})\
    .reset_index()\
    .pivot(index='geo', columns='mm', values='runs')\
    .reset_index()\
    .reindex(columns=['geo', *mm_order])\
    .sort_values(by='geo', key=lambda x: x.map({k: v for v, k in enumerate(geo_order)}))\
    .set_index('geo')

# display(mm_compat)


ipm_compat = compat.groupby(by=['geo', 'ipm'])\
    .agg({'runs': boolean_or})\
    .reset_index()\
    .pivot(index='geo', columns='ipm', values='runs')\
    .reset_index()\
    .reindex(columns=['geo', *ipm_order])\
    .sort_values(by='geo', key=lambda x: x.map({k: v for v, k in enumerate(geo_order)}))\
    .set_index('geo')

# display(ipm_compat)

palette = ['#AD0501', '#6EC16B']

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharey=True)

sns.heatmap(mm_compat,
            cmap=palette,
            cbar=False,
            square=True,
            linewidths=1,
            ax=ax1)

sns.heatmap(ipm_compat,
            cmap=palette,
            cbar=False,
            square=True,
            linewidths=1,
            ax=ax2)

plt.suptitle("Module compatibility")
plt.tight_layout()
plt.show()

# NOTE: and when I ran this, it matched my expectations as laid out in this spreadsheet:
# https://docs.google.com/spreadsheets/d/1FF_ovXYQUb65JagcLj8QCmCzs2_Pzd9qnZ5Cr28v4jw/edit?usp=sharing

In [ ]:
# What are the slowest sims (that succeed)?
compat\
    .loc[compat['runs'] == True]\
    .sort_values(by='runtime', ascending=False)\
    .reset_index(drop=True)